# Data Preprocessing
As of 3/10/2020, the [dataset](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2) provided by the city of Chicago on crime (excluding murders) contains over 7 millions rows and 22 columns. This [dataset](https://data.cityofchicago.org/Public-Safety/Homicides/k9xv-yxzs) contained the homicides, about 10,000, from the last 20 years. To facilitate early exploration of the data and focus on more recent, relevant trends, I removed crimes from before 2010, unneeded columns, and rows with nulls. The reduced dataset contained just under 3 million rows of crimes  
  
After reducing the size of the dataset, I cleaned up the text columns by manually matching values of each column with a smaller subset of categories in excel, mapped the Community Area ID's to their name and group (e.g. Community Area 8 maps to Near North Side and Central) based on [this](https://en.wikipedia.org/wiki/Community_areas_in_Chicago) Wikipedia page, and added some categorical columns based on the date of the crime.  

Also added in the Community Area population sizes from the [2010 Census](https://www.chicago.gov/content/dam/city/depts/zlup/Zoning_Main_Page/Publications/Census_2010_Community_Area_Profiles/Census_2010_and_2000_CA_Populations.pdf) to allow for an approximated Crimes/Homicides per Capita calculation. Unfortunately, this data isn't provided year over year and as of 3/30/2020, the 2020 Census isn't available, which is why I needed to use 2010 population sizes.

## Dataset Description:
These are the original column descriptions from the City of Chicago 
[website](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2)  

| Column Name  | Column Description |  
| :-:    | :-: |  
| ID           | Unique identifier for the record |
| Case Number  | The Chicago Police Department Records Division Number |
| Date         | Date when the incident occurred (sometimes an estimate) |
| Block        | The partially redacted address where the incident occurred, placing it on the same block as the actual address |
| IUCR         | Illinois Uniform Crime Reporting code |
| Primary Type | The primary description of the IUCR code |
| Description  | The secondary description of the IUCR code, a subcategory of the primary description |
| Location Description | Description of the location where the incident occurred |
| Arrest | Indicates whether an arrest was made |
| Domestic | Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence act |
| Beat | Indicates the beat where the incident occurred. A beat is the smallest police geographic area. 3 to 5 beats make up a police sector, and 3 sectors make up a police district |
| District | Indicates the police district where the incident occurred |
| Ward | The ward (City Council district) where the incident occurred |
| Community Area | Indicates the community area where the incident occurred (Chicago has 77 community areas) |
| FBI Code | Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS) |
| X Coordinate | The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block |
| Y Coordinate | The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block |
| Year | The year the incident occurred |
| Updated On | Date and time the record was last updated |
| Latitude | The latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block |
| Longitude | The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block |
| Location | The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block |

In [1]:
import os
import findspark
findspark.init()

#Setting the required parameters to start up PySpark
driver_memory = '6g'
num_executors = 2
executor_memory = '1g'
pyspark_submit_args = ' --driver-memory ' + driver_memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

spark = SparkSession.builder.master("local[*]").getOrCreate()

### Defining the Dataset Schemas

The schema for both the homicides and general crimes data is based on the metadata shown [here](https://data.cityofchicago.org/Public-Safety/Homicides/k9xv-yxzs).

In [3]:
crime_schema = StructType([
    StructField('id', IntegerType(), False),
    StructField('case_number', StringType(), True),
    StructField('date', StringType(), True),
    StructField('block', StringType(), True),
    StructField('iucr', StringType(), True),
    StructField('primary_type', StringType(), True),
    StructField('description', StringType(), True),
    StructField('location_description', StringType(), True),
    StructField('arrest', BooleanType(), True),
    StructField('domestic', BooleanType(), True),
    StructField('beat', StringType(), True),
    StructField('district', StringType(), True),
    StructField('ward', IntegerType(), True),
    StructField('community_area', StringType(), True),
    StructField('fbi_code', StringType(), True),
    StructField('x_coord', IntegerType(), True),
    StructField('y_coord', IntegerType(), True),
    StructField('year', IntegerType(), True),
    StructField('updated_on', DateType(), True),
    StructField('latitude', DoubleType(), True),
    StructField('longitude', DoubleType(), True),
    StructField('location', StringType(), True),
])

ca_schema = StructType([
    StructField('community_area', IntegerType(), False),
    StructField('name', StringType(), False),
    StructField('region', StringType(), False),
    StructField('pop_2010', IntegerType(), False),
])

loc_schema = StructType([
    StructField('location_description', StringType(), False),
    StructField('location_formatted', StringType(), False),
])

### Reading in the Datasets

In [4]:
# read in the homicides and non-homicide crimes and append the datasets together
homicides_df = spark.read.csv('./Data/Homicides.csv', schema=crime_schema, header=True)
gen_crimes_df = spark.read.csv('./Data/Crimes_-_2001_to_Present.csv', schema=crime_schema, header=True)
crime_df = homicides_df.union(gen_crimes_df)
# crime_df = spark.read.csv('./Data/Homicides.csv', schema=crime_schema, header=True) # for testing purposes

# read in the formatted location descriptions and the community area attributes
location_desc_df = spark.read.csv("./Data/Location.csv", schema=loc_schema, header=True)
comm_area_df = spark.read.csv("./Data/Community_Areas.csv", schema=ca_schema, header=True)

print("There were {:,d} crimes in Chicago from 2001 to the present.".format(crime_df.count()))

There were 7,233,087 crimes in Chicago from 2001 to the present.


### Using SQL to Create the Final Dataset

In [5]:
# Converting `date` from String to Timestamp
crime_df = crime_df.withColumn("date", F.to_timestamp("date", "MM/dd/yyyy hh:mm:ss a"))
crime_df = crime_df.withColumn("week_day_num", F.dayofweek(crime_df.date))

crime_df.createOrReplaceTempView("crime")
location_desc_df.createOrReplaceTempView("location")
comm_area_df.createOrReplaceTempView("comm_area")

In [6]:
query = """
SELECT crime.date AS Date
    ,CASE WHEN (crime.primary_type = 'NON - CRIMINAL') OR (crime.primary_type = 'NON-CRIMINAL (SUBJECT SPECIFIED)') 
        THEN 'NON-CRIMINAL' ELSE crime.primary_type END AS `Primary Type`
    ,CAST(crime.arrest as INTEGER) AS Arrest
    ,crime.domestic AS Domestic
    ,YEAR(crime.date) AS Year
    ,crime.latitude AS Latitude
    ,crime.longitude AS Longitude
    ,location.location_formatted AS `Location Description`
    ,comm_area.name AS `Community Area`
    ,comm_area.region AS Region
    ,comm_area.pop_2010 AS `2010 Population`
    ,MONTH(crime.date) AS Month
    ,CASE WHEN HOUR(crime.date) BETWEEN  5 AND 12 THEN 'Morning'
          WHEN HOUR(crime.date) BETWEEN 12 AND 17 THEN 'Afternoon'
          WHEN HOUR(crime.date) BETWEEN 17 AND 20 THEN 'Evening'
          WHEN HOUR(crime.date) >= 20 OR HOUR(crime.date) < 5 THEN 'Night'
     END AS `Time of Day`

    ,CASE WHEN crime.week_day_num = 1 THEN 'Sunday'
          WHEN crime.week_day_num = 2 THEN 'Monday'
          WHEN crime.week_day_num = 3 THEN 'Tuesday'
          WHEN crime.week_day_num = 4 THEN 'Wednesday'
          WHEN crime.week_day_num = 5 THEN 'Thursday'
          WHEN crime.week_day_num = 6 THEN 'Friday'
          WHEN crime.week_day_num = 7 THEN 'Saturday'
     END AS `Day of Week`

    ,CASE WHEN MONTH(crime.date) IN ( 3,  4,  5) THEN 'Spring'
          WHEN MONTH(crime.date) IN ( 6,  7,  8) THEN 'Summer'
          WHEN MONTH(crime.date) IN ( 9, 10, 11) THEN 'Fall'
          WHEN MONTH(crime.date) IN (12,  1,  2) THEN 'Winter'
     END AS Season

FROM crime
    INNER JOIN location ON location.location_description = crime.location_description
    INNER JOIN comm_area ON comm_area.community_area = crime.community_area

WHERE YEAR(crime.date) BETWEEN 2010 AND 2019
    AND crime.date IS NOT NULL
    AND crime.latitude IS NOT NULL
    AND crime.arrest IS NOT NULL
"""

In [9]:
%%timeit -n1 -r1 # to let you know how long it takes this cell to run

crime_sql_df = spark.sql(query)

print("Number of crimes in the final dataset: {:,d}".format(crime_sql_df.count()))

# convert the crime type to title case for nicer looking values
crime_sql_df = crime_sql_df.withColumn('Primary Type', F.initcap(crime_sql_df['Primary Type']))

# using collect() and then converting to pandas df because it's faster
crime_dict = crime_sql_df.collect()
crime_pd_df = pd.DataFrame(crime_dict, columns=crime_sql_df.columns)
crime_pd_df.to_csv("./Data/crimes_cleaned.csv", index=False)

Number of crimes in the final dataset: 2,949,016
56.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
